In [ ]:
# Importing necessary libraries
import numpy as np
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Mounting Google Drive
drive.mount('/content/drive')

# Define image dimensions and batch size
IMG_HEIGHT = 150
IMG_WIDTH = 150
BATCH_SIZE = 32

# Define paths for training and validation data
train_dir = '/content/drive/My Drive/Pneumonia Detection/train'
val_dir = '/content/drive/My Drive/Pneumonia Detection/val'

# Defining ImageDataGenerator for augmenting the training data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Defining ImageDataGenerator for validating the validation data
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Setting up the training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary')

# Setting up the validation data generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary')

# Creating the CNN model
model = Sequential()

# Adding the first convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(MaxPooling2D((2, 2)))

# Adding the second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Adding the third convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flattening the output from the last convolutional layer
model.add(Flatten())

# Adding the first dense layer with 512 units and dropout
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

# Adding the second dense layer with 256 units and dropout
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Adding the output layer with sigmoid activation
model.add(Dense(1, activation='sigmoid'))

# Compiling the model with binary crossentropy loss and Adam optimizer
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Fitting the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=50,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE)

# Evaluating the model
_, accuracy = model.evaluate(val_generator, steps=val_generator.samples // BATCH_SIZE)
print('Accuracy on validation set:', accuracy)
